In [3]:
import autogen
import math
from typing_extensions import Annotated

llm_config = {
    "config_list": [{ "base_url":"http://localhost:1234/v1",
                      "model":"llama3.1:8b",
                      "api_key":"not_used" }],
    "timeout": 120,
    "cache_seed": None,
}

In [9]:
user_proxy = autogen.UserProxyAgent(
    name="Interface with the user and code executor",
    is_termination_msg=(lambda msg: "conclude" in msg["content"].lower()),
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False},
    max_consecutive_auto_reply=2,
)

domain_expert = autogen.AssistantAgent(
    name="Domain expert",
    llm_config=llm_config,
    system_message="""
    If you have to compute a mathematical function, like a square root, always use the the related function, if available.
    When you think that the achieved result is satisfactory, write 'CONCLUDED'.
    """
)

In [10]:
@user_proxy.register_for_execution()
@domain_expert.register_for_llm(description="Function to compute a square root.")
def square_root_calculator(
        number: Annotated[float, "Number of which the square root is to be computed"],
    ) -> str:
    number = float(number)
    if number >= 0:
        return f"The square root of {number} is {math.sqrt(number):.4f}"
    raise ValueError(f"The square root of the negative number {number} cannot be computed")

In [11]:
domain_expert.llm_config["tools"] # check the JSON configuration

[{'type': 'function',
  'function': {'description': 'Function to compute a square root.',
   'name': 'square_root_calculator',
   'parameters': {'type': 'object',
    'properties': {'number': {'type': 'number',
      'description': 'Number of which the square root is to be computed'}},
    'required': ['number']}}}]

In [12]:
domain_expert.reset()
res = user_proxy.initiate_chat(domain_expert, message="What is the square root of 1234.56789?")

Interface with the user and code executor (to Domain expert):

What is the square root of 1234.56789?

--------------------------------------------------------------------------------
[autogen.oai.client: 10-13 17:09:37] {409} WARNING - Model llama3.1:8b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
Domain expert (to Interface with the user and code executor):


***** Suggested tool call (call_8dzfbe8a): square_root_calculator *****
Arguments: 
{"number":"1234.56789"}
***********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION square_root_calculator...
Interface with the user and code executor (to Domain expert):

Interface with the user and code executor (to Domain expert):

***** Response from calling tool (call_8dzfbe8a) *****
The square root of 1234.56789

In [13]:
from pprint import pprint
pprint(res.chat_history)

[{'content': 'What is the square root of 1234.56789?',
  'name': 'Interface with the user and code executor',
  'role': 'assistant'},
 {'content': '',
  'role': 'assistant',
  'tool_calls': [{'function': {'arguments': '{"number":"1234.56789"}',
                               'name': 'square_root_calculator'},
                  'id': 'call_8dzfbe8a',
                  'type': 'function'}]},
 {'content': 'The square root of 1234.56789 is 35.1364',
  'name': 'Interface with the user and code executor',
  'role': 'tool',
  'tool_responses': [{'content': 'The square root of 1234.56789 is 35.1364',
                      'role': 'tool',
                      'tool_call_id': 'call_8dzfbe8a'}]},
 {'content': 'CONCLUDED.\n'
             '\n'
             'Note: I used the `math.sqrt()` function in Python to compute the '
             'square root.',
  'name': 'Domain expert',
  'role': 'user'}]
